Training model with MNIST data

Jeep Naarkom  7/15/2023

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

#from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, activations
from tensorflow.keras import Model, layers

TensorFlow version: 2.12.0


In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

11490434/11490434 [==============================] - 2s 0us/step


In [3]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

Layer notes

*   convolution 5x5 kernal + 2 pad 6 filter 28x28x6    (
*   sigmoid 1:1  28x28x6
*   pool 2x2 ave 2 stride 14x14x6
*   convolution 5x5 kernal + 2 pad 16 filter 10x10x6
*   sigmoid 1:1  10x10x6
*   pool 2x2 ave 2 stride 5x5x16
*   flatten 400
*   sigmoid fully connect 120
*   sigmoid fully connect 84
*   sigmoid fully connect 10



In [4]:
def sigmoid_1to1(x):
    return 1 / (1 + tf.exp(-x))

def lenetModel():
    model = tf.keras.models.Sequential([

    tf.keras.layers.Conv2D(6, kernel_size=(5, 5), padding = "same", activation='sigmoid', input_shape=(28, 28, 1)),
    tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2)),
    tf.keras.layers.Conv2D(16, kernel_size=(5, 5), activation='sigmoid', input_shape=(10, 10, 1)),
    tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(120, activation='sigmoid'),
    tf.keras.layers.Dense(84, activation='sigmoid'),
    tf.keras.layers.Dense(10, activation='sigmoid')
    ])
    return model

# Create an instance of the LeNet-5 model
model = lenetModel()


In [5]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [6]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [7]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [8]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [9]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

/usr/local/lib/python3.10/dist-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Epoch 1, Loss: 0.7361156344413757, Accuracy: 76.29499816894531, Test Loss: 0.21620838344097137, Test Accuracy: 93.44000244140625
Epoch 2, Loss: 0.17777234315872192, Accuracy: 94.5616683959961, Test Loss: 0.12037143111228943, Test Accuracy: 96.2300033569336
Epoch 3, Loss: 0.11364836245775223, Accuracy: 96.51000213623047, Test Loss: 0.10422803461551666, Test Accuracy: 96.75
Epoch 4, Loss: 0.08400199562311172, Accuracy: 97.4433364868164, Test Loss: 0.07318657636642456, Test Accuracy: 97.63999938964844
Epoch 5, Loss: 0.06881289184093475, Accuracy: 97.8516616821289, Test Loss: 0.05711270123720169, Test Accuracy: 98.1199951171875
